In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import sys
import os
import xlrd
import io

In [2]:
def findCurrencyChinese(marketcap):
    reChinese = re.compile('[\u4e00-\u9fa5]+') 
    return reChinese.findall(marketcap)[0]

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def currency_num_formatter(num, w=10000, y=100000000):
    num_unit = ''
    if int(num/y) > 1:
        num = float(num/y)
        num_unit = '亿'
    elif int(num/w) > 1:
        num = float(num/w)
        num_unit = '万'
    else:
        num = num
        num_unit = ''
    return ('%.2f' + num_unit) % num

In [57]:
### settings by user

### range setting
etf_category_toggle = 'PRCfi'
# etf_category_toggle = 'PRCstock'
# etf_category_toggle = 'PRCcomm'
# etf_category_toggle = 'PRCcurr'
# etf_category_toggle = 'hkfi'
# etf_category_toggle = 'hkstock'
# etf_category_toggle = 'hkcomm'
# etf_category_toggle = 'hkcurr'
# etf_category_toggle = 'hklip'

source_csv_date = '20181128'

In [4]:
raw_info = pd.read_csv("product-brief-raw-prc+hk-v%s.csv" % source_csv_date) 
raw_info = raw_info.replace(np.nan, '', regex=True)
# raw_info

df_columns = ['code', 'region', 'description']
etf_desc_df = pd.DataFrame(columns=df_columns)

In [58]:
# PRC ETF description generation: will generate 510820.SH. need to delete
print("================ %s ===============" % etf_category_toggle)

raw_info_filtered = raw_info[raw_info['region'] == etf_category_toggle]

for i in range(len(raw_info_filtered)):
    entry = raw_info_filtered.iloc[i, :]
    
    try:
        if entry['region'].startswith("PRC"):

            if entry['suspend_or_pending'] == 'pending':
                marketcap_statement = "截至2018年11月28日，该基金暂未上市。"
                listing_date_statement = ""
                
            else:
                marketcap_statement = "截至2018年11月28日，" + entry['full_name'] + "市值为" + currency_num_formatter(float(entry['market_cap'].replace(",", ""))) + \
                "元人民币，过去22日日均交易额为" + currency_num_formatter(float(entry['avg_daily trading vol'].replace(",", ""))) + "元人民币。"
                
                listing_date_dtobj = datetime.strptime(entry['set-up_date'], '%Y/%m/%d')                
                listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日上市，"
                
            product_brief_header = color.BOLD + entry['full_name'].replace("交易型开放式指数证券投资基金", "ETF") + "（" + entry['code'] + "）" + color.END + "\n"
            product_brief = "该基金" + \
            listing_date_statement + \
            "进行" + \
            entry['passive/active'] + "投资，追踪" + \
            entry['underlying_index_short_stripped_for_use'] + "。" + \
            entry['index_brief'] + "该基金为" + \
            entry['physical/synthetic'] + "ETF。" + \
            "该基金管理人为" + entry['manager'] + \
            "，管理费为" + "%.2f" % (float(entry['mgt_fee'])) + \
            "%；基金托管人为" + entry['custodian'] + \
            "，托管费为" + "%.2f" % (float(entry['custodian_fee'])) + "%。" + \
            marketcap_statement + \
            entry['remarks'] + \
            "\n"

            print(product_brief_header + product_brief)
            
            df_tmp = pd.DataFrame([[raw_info_filtered.iloc[i, 1], etf_category_toggle, product_brief]], columns = df_columns)
            etf_desc_df = etf_desc_df.append(df_tmp)
            
        
    except:
        print(entry['code'], " error-----")
    

================ PRCfi ===============
国泰上证10年期国债ETF（511260.SH）
该基金2017年8月24日上市，进行被动式投资，追踪上证10年期国债指数。上证10年期国债指数的样本由剩余期限7到10年且在上交所挂牌的国债组成。该基金为实物资产ETF。该基金管理人为国泰基金管理有限公司，管理费为0.30%；基金托管人为中国建设银行股份有限公司，托管费为0.10%。截至2018年11月28日，国泰上证10年期国债ETF市值为19304.23万元人民币，过去22日日均交易额为8021.31万元人民币。

广发上证10年期国债ETF（511290.SH）
该基金2018年4月27日上市，进行被动式投资，追踪上证10年期国债指数。上证10年期国债指数的样本由剩余期限7到10年且在上交所挂牌的国债组成。该基金为实物资产ETF。该基金管理人为广发基金管理有限公司，管理费为0.30%；基金托管人为中国工商银行股份有限公司，托管费为0.10%。截至2018年11月28日，广发上证10年期国债ETF市值为5392.03万元人民币，过去22日日均交易额为600.23元人民币。

富国中证10年期国债ETF（511310.SH）
该基金2018年4月26日上市，进行被动式投资，追踪中证10年期国债指数。中证10年期国债指数的样本券由同时在银行间和上海证券交易所市场挂牌、且剩余期限位于8.5到10年之间的国债组成。该基金为实物资产ETF。该基金管理人为富国基金管理有限公司，管理费为0.25%；基金托管人为中国工商银行股份有限公司，托管费为0.05%。截至2018年11月28日，富国中证10年期国债ETF市值为2656.50万元人民币，过去22日日均交易额为16.90万元人民币。

国泰上证5年期国债ETF（511010.SH）
该基金2013年3月25日上市，进行被动式投资，追踪上证5年期国债指数。上证5年期国债指数的样本由剩余期限4到7年且在上交所挂牌的国债组成。该基金为实物资产ETF。该基金管理人为国泰基金管理有限公司，管理费为0.30%；基金托管人为中国建设银行股份有限公司，托管费为0.10%。截至2018年11月28日，国泰上证5年期国债ETF市值为3.21亿元人民币，过去22日日均交易额为2.30亿元人民币。

嘉实中证金边中期国债E

In [6]:
# Special case - PRC for 510820.SH - suspended purchase & redemption

entry = raw_info[raw_info['code'] == '510820.SH'].T.squeeze()

try:
    if entry['region'].startswith("PRC"):

        if entry['suspend_or_pending'] == 'pending':
            listing_date_statement = ""

        else:            
            listing_date_dtobj = datetime.strptime(entry['set-up_date'], '%Y/%m/%d')                
            listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日上市，"

        product_brief_header = color.BOLD + entry['full_name'].replace("交易型开放式指数发起式证券投资基金", "ETF") + "（" + entry['code'] + "）" + color.END + "\n"
        product_brief = "该基金" + \
        listing_date_statement + \
        "进行" + \
        entry['passive/active'] + "投资，追踪" + \
        entry['underlying_index_short_stripped_for_use'] + "。" + \
        entry['index_brief'] + "该基金为" + \
        entry['physical/synthetic'] + "ETF。" + \
        "该基金管理人为" + entry['manager'] + \
        "，基金托管人为" + entry['custodian'] + \
        "。为保护投资者权益、维护基金的稳定运作，该基金于2018年3月2日起暂停申购、赎回，至发稿日暂未恢复。"

        print(product_brief_header + product_brief)
        
        df_tmp = pd.DataFrame([['510820.SH', etf_category_toggle, product_brief]], columns = df_columns)
        etf_desc_df = etf_desc_df.append(df_tmp)
        

except Exception as e:
    print(entry['code'], " error-----", e)

上证上海改革发展主题ETF（510820.SH）
该基金2017年12月1日上市，进行被动式投资，追踪上海改革指数。上证改革发展主题指数以本地股为主要选样空间，在“一带一路”、“四个中心”、“自贸区建设”等战略的基础上，结合国企改革、产业升级、重点项目的推进情况，从相关受益板块中选取最具区域主题代表性的股票组成指数样本股，反映上海主题在A股的整体走势。该基金为实物资产ETF。该基金管理人为汇添富基金管理股份有限公司，基金托管人为中国工商银行股份有限公司。为保护投资者权益、维护基金的稳定运作，该基金于2018年3月2日起暂停申购、赎回，至发稿日暂未恢复。


In [42]:
# HK ETF description generation

print("================ %s ===============" % etf_category_toggle)
hk_code_all_ls = raw_info.loc[raw_info['region'] == etf_category_toggle]['code'].tolist()

hk_code_cnh_counter_ls = [code for code in hk_code_all_ls if code.startswith("8")]
hk_code_cnh_counter_ls_copy = hk_code_cnh_counter_ls.copy()

hk_code_usd_counter_ls = [code for code in hk_code_all_ls if code.startswith("9")]
hk_code_usd_counter_ls_copy = hk_code_usd_counter_ls.copy()

hk_code_hkd_counter_ls = [code for code in hk_code_all_ls if code not in hk_code_usd_counter_ls and code not in hk_code_cnh_counter_ls]
# hk_code_hkd_counter_ls

# for i in range(3, 4):
for i in range(len(hk_code_hkd_counter_ls)):
    code_hkd = hk_code_hkd_counter_ls[i]
    entry = raw_info.loc[raw_info['code'] == code_hkd].T.squeeze()
    
    try:            
        # currency counter toggle
        code_cnh = "8" + code_hkd
        code_usd = "9" + code_hkd[1:]   
        
        # suspension toggle
        if entry['suspend_or_pending'] == 'suspended':
            suspension_dtobj = datetime.strptime(entry['suspension_date'], '%m/%d/%y')
            marketcap_statement = "该基金已于" + str(suspension_dtobj.year) + "年" + str(suspension_dtobj.month) + "月" + str(suspension_dtobj.day) + "日停牌。"
            listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日上市，"  
            ongoing_exp_statement = ""
            
            if code_cnh in hk_code_cnh_counter_ls:            
                code_cnh = "/" + code_cnh
            else:
                code_cnh = "" 
                
            if code_usd in hk_code_usd_counter_ls:            
                code_usd = "/" + code_usd
            else:
                code_usd = "" 
                
        else:
            hkd_flag = '港币'
            
            hkd_marketcap = raw_info.loc[raw_info['code'] == code_hkd].T.squeeze()['market_cap']
#             hkd_number_mn = "%.2f" % (float(hkd_marketcap[:hkd_marketcap.find("港币")].replace(",", ""))/1000000)
            hkd_marketcap_num = currency_num_formatter(float(hkd_marketcap[:hkd_marketcap.find("港币")].replace(",", "")))
            hkd_avg_TO = raw_info.loc[raw_info['code'] == code_hkd].T.squeeze()['avg_daily trading vol']
            hkd_avg_TO_num = currency_num_formatter(float(hkd_avg_TO[:hkd_avg_TO.find("港币")].replace(",", "")))

            if code_cnh in hk_code_cnh_counter_ls:            
            
                hk_code_cnh_counter_ls_copy.remove(code_cnh)

                cnh_counter_statement = "；另有" + code_cnh + "人民币柜台，以人民币结算"
                cnh_marketcap = raw_info.loc[raw_info['code'] == code_cnh].T.squeeze()['market_cap']
                cnh_marketcap_num = currency_num_formatter(float(cnh_marketcap[:cnh_marketcap.find("人民币")].replace(",", "")))
                cnh_avg_TO = raw_info.loc[raw_info['code'] == code_cnh].T.squeeze()['avg_daily trading vol']
                cnh_avg_TO_num = currency_num_formatter(float(cnh_avg_TO[:cnh_avg_TO.find("人民币")].replace(",", "")))

                cnh_flag = '人民币'

                cnh_marketcap_statement = "，人民币柜台（" + code_cnh + "）过去22天日均交易额" + cnh_avg_TO_num + cnh_flag
                code_cnh = "/" + code_cnh
            else:
                cnh_counter_statement = ""
                cnh_marketcap_statement = ""
                code_cnh = ""

            if code_usd in hk_code_usd_counter_ls:

                hk_code_usd_counter_ls_copy.remove(code_usd)

                usd_counter_statement = "；以及" + code_usd + "美元柜台，以美元结算"
                usd_marketcap = raw_info.loc[raw_info['code'] == code_usd].T.squeeze()['market_cap']
                usd_marketcap_num = currency_num_formatter(float(usd_marketcap[:usd_marketcap.find("美元")].replace(",", "")))
                usd_avg_TO = raw_info.loc[raw_info['code'] == code_usd].T.squeeze()['avg_daily trading vol']
                usd_avg_TO_num = currency_num_formatter(float(usd_avg_TO[:usd_avg_TO.find("美元")].replace(",", "")))

                usd_flag = '美元'

                usd_marketcap_statement = "，美元柜台（" + code_usd + "）过去22天日均交易额" + usd_avg_TO_num + usd_flag
                code_usd = "/" + code_usd            
            else:
                usd_counter_statement = ""
                usd_marketcap_statement = ""
                code_usd = ""       
                
            marketcap_statement = "截至2018年11月28日，" + entry['full_name'] + "市值为" + hkd_marketcap_num + hkd_flag + \
            "，港币柜台（" + code_hkd + "）过去22日日均交易额为" + hkd_avg_TO_num + hkd_flag + cnh_marketcap_statement + usd_marketcap_statement + "。"
            
            listing_date_dtobj = datetime.strptime(entry['set-up_date'], '%Y/%m/%d')                
            listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日成立，"
            ongoing_exp_statement = "该基金总开支比率为" + "%.2f" % (float(entry['ongoing_fee'])) + "%。"
            counter_statement = code_hkd + "以港币结算" + cnh_counter_statement + usd_counter_statement

        # custodian toggle
        if entry['custodian'] == "无":
            mgr_and_custodian_statement = "该基金管理人为" + entry['manager'] + "，无基金托管人。"
        else:
            mgr_and_custodian_statement = "该基金管理人为" + entry['manager'] + "，基金托管人为" + entry['custodian'] + "。"
        
        product_brief_header = color.BOLD + entry['full_name'] + "（" + code_hkd + code_cnh + code_usd + "）" + color.END + "\n" 
        
        product_brief = "该基金" + \
        listing_date_statement + \
        "进行" + \
        entry['passive/active'].strip() + "投资，追踪" + \
        entry['underlying_index_short_stripped_for_use'] + "。" + \
        entry['index_brief'] + "该基金为" + \
        entry['physical/synthetic'] + "ETF。" + \
        counter_statement + "。" + \
        marketcap_statement + \
        mgr_and_custodian_statement + \
        ongoing_exp_statement + \
        entry['remarks'] + \
        "\n"        
        
        print(product_brief_header + product_brief)
        
        df_tmp = pd.DataFrame([[code_hkd, etf_category_toggle, product_brief]], columns = df_columns)
        etf_desc_df = etf_desc_df.append(df_tmp)
        
        if code_cnh[1:] in hk_code_cnh_counter_ls:
        
            df_tmp = pd.DataFrame([[code_cnh[1:], etf_category_toggle, product_brief]], columns = df_columns)
            etf_desc_df = etf_desc_df.append(df_tmp)
        
        if code_usd[1:] in hk_code_usd_counter_ls:
        
            df_tmp = pd.DataFrame([[code_usd[1:], etf_category_toggle, product_brief]], columns = df_columns)
            etf_desc_df = etf_desc_df.append(df_tmp)

    except Exception as e:
        print(entry['code'], " error-----", e)
    

================ hkcurr ===============
南方东英港元货币市场ETF（3053.HK/83053.HK）
该基金2018年7月18日成立，进行被动式投资，追踪三个月期港元利息结算利率。该基金为实物资产ETF。3053.HK以港币结算；另有83053.HK人民币柜台，以人民币结算。截至2018年11月28日，南方东英港元货币市场ETF市值为24.55亿港币，港币柜台（3053.HK）过去22日日均交易额为1616.13万港币，人民币柜台（83053.HK）过去22天日均交易额16.20万人民币。该基金管理人为南方东英资产管理有限公司，无基金托管人。该基金总开支比率为0.50%。



In [22]:
# HK ETF special case: base currency USD not starting with "9": ['2821.HK', '7261.HK', '7331.HK', '7322.HK']

print("================ %s ===============" % etf_category_toggle)

for code_hkd in ['7261.HK', '7331.HK', '7322.HK']:
    entry = raw_info.loc[raw_info['code'] == code_hkd].T.squeeze()

    try:            
        # currency counter toggle
        code_cnh = "8" + code_hkd
        code_usd = "9" + code_hkd[1:]

        if code_cnh in hk_code_cnh_counter_ls:

            hk_code_cnh_counter_ls_copy.remove(code_cnh)

            cnh_counter_statement = "；另有" + code_cnh + "人民币柜台，以人民币结算"
            cnh_marketcap = raw_info.loc[raw_info['code'] == code_cnh].T.squeeze()['market_cap']
            cnh_marketcap_num = currency_num_formatter(float(cnh_marketcap[:cnh_marketcap.find("人民币")].replace(",", "")))
            cnh_avg_TO = raw_info.loc[raw_info['code'] == code_cnh].T.squeeze()['avg_daily trading vol']
            cnh_avg_TO_num = currency_num_formatter(float(cnh_avg_TO[:cnh_avg_TO.find("人民币")].replace(",", "")))

            cnh_flag = '人民币'

            cnh_marketcap_statement = "，人民币柜台（" + code_cnh + "）过去22天日均交易额" + cnh_avg_TO_num + cnh_flag
            code_cnh = "/" + code_cnh
        else:
            cnh_counter_statement = ""
            cnh_marketcap_statement = ""
            code_cnh = ""

        if code_usd in hk_code_usd_counter_ls:

            hk_code_usd_counter_ls_copy.remove(code_usd)

            usd_counter_statement = "；以及" + code_usd + "美元柜台，以美元结算"
            usd_marketcap = raw_info.loc[raw_info['code'] == code_usd].T.squeeze()['market_cap']
            usd_marketcap_num = currency_num_formatter(float(usd_marketcap[:usd_marketcap.find("美元")].replace(",", "")))
            usd_avg_TO = raw_info.loc[raw_info['code'] == code_usd].T.squeeze()['avg_daily trading vol']
            usd_avg_TO_num = currency_num_formatter(float(usd_avg_TO[:usd_avg_TO.find("美元")].replace(",", "")))

            usd_flag = '美元'

            usd_marketcap_statement = "，美元柜台（" + code_usd + "）过去22天日均交易额" + usd_avg_TO_num + usd_flag
            code_usd = "/" + code_usd            
        else:
            usd_counter_statement = ""
            usd_marketcap_statement = ""
            code_usd = ""            

        counter_statement = code_hkd + "以美元结算" + cnh_counter_statement + usd_counter_statement

        # suspension toggle
        if entry['suspend_or_pending'] == 'suspended':
            suspension_dtobj = datetime.strptime(entry['suspension_date'], '%m/%d/%y')
            marketcap_statement = "该基金已于" + str(suspension_dtobj.year) + "年" + str(suspension_dtobj.month) + "月" + str(suspension_dtobj.day) + "日停牌。"
            listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日成立，"  
            ongoing_exp_statement = ""
        else:
            hkd_flag = '港币'

            hkd_marketcap = raw_info.loc[raw_info['code'] == code_hkd].T.squeeze()['market_cap']
    #             hkd_number_mn = "%.2f" % (float(hkd_marketcap[:hkd_marketcap.find("港币")].replace(",", ""))/1000000)
            hkd_marketcap_num = currency_num_formatter(float(hkd_marketcap[:hkd_marketcap.find("美元")].replace(",", "")))
            hkd_avg_TO = raw_info.loc[raw_info['code'] == code_hkd].T.squeeze()['avg_daily trading vol']
            hkd_avg_TO_num = currency_num_formatter(float(hkd_avg_TO[:hkd_avg_TO.find("美元")].replace(",", "")))

            marketcap_statement = "截至2018年11月28日，" + entry['full_name'] + "市值为" + hkd_marketcap_num + usd_flag + \
            "，过去22日日均交易额为" + hkd_avg_TO_num + usd_flag + cnh_marketcap_statement + usd_marketcap_statement + "。"

            listing_date_dtobj = datetime.strptime(entry['set-up_date'], '%Y/%m/%d')                
            listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日成立，"
            ongoing_exp_statement = "该基金总开支比率为" + "%.2f" % (float(entry['ongoing_fee'])) + "%。"

        # custodian toggle
        if entry['custodian'] == "无":
            mgr_and_custodian_statement = "该基金管理人为" + entry['manager'] + "，无基金托管人。"
        else:
            mgr_and_custodian_statement = "该基金管理人为" + entry['manager'] + "，基金托管人为" + entry['custodian'] + "。"

        product_brief_header = color.BOLD + entry['full_name'] + "（" + code_hkd + code_cnh + code_usd + "）" + color.END + "\n" 

        product_brief = "该基金" + \
        listing_date_statement + \
        "进行" + \
        entry['passive/active'].strip() + "投资，追踪" + \
        entry['underlying_index_short_stripped_for_use'] + "。" + \
        entry['index_brief'] + "该基金为" + \
        entry['physical/synthetic'] + "ETF。" + \
        counter_statement + "。" + \
        marketcap_statement + \
        mgr_and_custodian_statement + \
        ongoing_exp_statement + \
        entry['remarks'] + \
        "\n"        

        print(product_brief_header + product_brief)

        df_tmp = pd.DataFrame([[code_hkd, etf_category_toggle, product_brief]], columns = df_columns)
        etf_desc_df = etf_desc_df.append(df_tmp)

        if code_cnh[1:] in hk_code_cnh_counter_ls:

            df_tmp = pd.DataFrame([[code_cnh[1:], etf_category_toggle, product_brief]], columns = df_columns)
            etf_desc_df = etf_desc_df.append(df_tmp)

        if code_usd[1:] in hk_code_usd_counter_ls:

            df_tmp = pd.DataFrame([[code_usd[1:], etf_category_toggle, product_brief]], columns = df_columns)
            etf_desc_df = etf_desc_df.append(df_tmp)

    except Exception as e:
        print(entry['code'], " error-----", e)
    

================ hklip ===============
华夏Direxion纳斯达克100指数每日杠杆（2x）产品（7261.HK）
该基金2016年9月28日成立，进行被动式投资，追踪Nasdaq100指数。纳斯达克100指数包括在纳斯达克股票市场上市的100家按市值计最大的国内和国际非金融公司。该指数反映了主要行业组织的公司，包括计算机硬件和软件、电信、零售/批发贸易和生物技术。纳斯达克100指数不包含金融公司（包括投资公司）的证券。该基金为合成ETF。7261.HK以美元结算。截至2018年11月28日，华夏Direxion纳斯达克100指数每日杠杆（2x）产品市值为6049.00万美元，过去22日日均交易额为94.12万美元。该基金管理人为华夏基金（香港）有限公司，无基金托管人。该基金总开支比率为2.32%。

华夏Direxion纳斯达克100指数每日反向（-1x）产品（7331.HK）
该基金2016年9月28日成立，进行被动式投资，追踪Nasdaq100指数。纳斯达克100指数包括在纳斯达克股票市场上市的100家按市值计最大的国内和国际非金融公司。该指数反映了主要行业组织的公司，包括计算机硬件和软件、电信、零售/批发贸易和生物技术。纳斯达克100指数不包含金融公司（包括投资公司）的证券。该基金为合成ETF。7331.HK以美元结算。截至2018年11月28日，华夏Direxion纳斯达克100指数每日反向（-1x）产品市值为2.03亿美元，过去22日日均交易额为965.99万美元。该基金管理人为华夏基金（香港）有限公司，无基金托管人。该基金总开支比率为3.00%。

未来资产标普500 每日反向（-1x）产品（7322.HK）
该基金2016年10月11日成立，进行被动式投资，追踪标准普尔500指数。标准普尔500®被广泛认为是美国大盘股的最佳单一指标。对标该指数的资产超过7.8万亿美元，其中指数资产约占2.2万亿美元。该指数包括500家领先公司，并覆盖大约80％的流通市值。该基金为合成ETF。7322.HK以美元结算。截至2018年11月28日，未来资产标普500 每日反向（-1x）产品市值为16815.00万美元，过去22日日均交易额为384.74万美元。该基金管理人为未来资产环球投资(香港)有限公司，无基金托管人。该基金总开

In [28]:
print("======== CNH & USD counters left over ======== \n", hk_code_cnh_counter_ls_copy, hk_code_usd_counter_ls_copy)

======== CNH & USD counters left over ======== 
 ['83168.HK'] []


In [29]:
# only cnh counter left over in commodity (20181127)
for i in range(len(hk_code_cnh_counter_ls_copy)):
    code_cnh = hk_code_cnh_counter_ls_copy[i]
    entry = raw_info.loc[raw_info['code'] == code_cnh].T.squeeze()
    
    try: 
        
        counter_statement = code_cnh + "仅有人民币柜台，以人民币结算" 
        
        # suspension toggle
        if entry['suspend_or_pending'] == 'suspended':
            suspension_dtobj = datetime.strptime(entry['suspension_date'], '%m/%d/%y')
            marketcap_statement = "该基金已于" + str(suspension_dtobj.year) + "年" + str(suspension_dtobj.month) + "月" + str(suspension_dtobj.day) + "日停牌。"
            listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日成立，"  
            ongoing_exp_statement = ""
        else:
            cnh_flag = '人民币'
            
            cnh_marketcap = raw_info.loc[raw_info['code'] == code_cnh].T.squeeze()['market_cap']
            cnh_marketcap_num = currency_num_formatter(float(cnh_marketcap[:cnh_marketcap.find("人民币")].replace(",", "")))
            cnh_avg_TO = raw_info.loc[raw_info['code'] == code_cnh].T.squeeze()['avg_daily trading vol']
            cnh_avg_TO_num = currency_num_formatter(float(cnh_avg_TO[:cnh_avg_TO.find("人民币")].replace(",", "")))
            
            marketcap_statement = "截至2018年11月28日，" + entry['full_name'] + "市值为" + cnh_marketcap_num + cnh_flag + \
            "，过去22日日均交易额为" + cnh_avg_TO_num + cnh_flag + cnh_marketcap_statement + "。"
            
            listing_date_dtobj = datetime.strptime(entry['set-up_date'], '%Y/%m/%d')                
            listing_date_statement = str(listing_date_dtobj.year) + "年" + str(listing_date_dtobj.month) + "月" + str(listing_date_dtobj.day) + "日成立，"
            ongoing_exp_statement = "该基金总开支比率为" + "%.2f" % (float(entry['ongoing_fee'])) + "%。"

        # custodian toggle
        if entry['custodian'] == "无":
            mgr_and_custodian_statement = "该基金管理人为" + entry['manager'] + "，无基金托管人。"
        else:
            mgr_and_custodian_statement = "该基金管理人为" + entry['manager'] + "，基金托管人为" + entry['custodian'] + "。"
        
        product_brief_header = color.BOLD + entry['full_name'] + "（" + code_cnh + "）" + color.END + "\n" 
        
        product_brief = "该基金" + \
        listing_date_statement + \
        "进行" + \
        entry['passive/active'].strip() + "投资，追踪" + \
        entry['underlying_index_short_stripped_for_use'] + "。" + \
        entry['index_brief'] + "该基金为" + \
        entry['physical/synthetic'] + "ETF。" + \
        counter_statement + "。" + \
        marketcap_statement + \
        mgr_and_custodian_statement + \
        ongoing_exp_statement + \
        entry['remarks'] + \
        "\n"        
        
        print(product_brief_header + product_brief)
        
        df_tmp = pd.DataFrame([[code_cnh, etf_category_toggle, product_brief]], columns = df_columns)
        etf_desc_df = etf_desc_df.append(df_tmp)
        
    except Exception as e:
        print(entry['code'], " error-----", e)
        

恒生人民币黄金ETF（83168.HK）
该基金2012年2月14日成立，进行被动式投资，追踪伦敦黄金定盘价（早盘）。伦敦黄金定盘价为ICE Benchmark Administration Limited（“IBA”）于拍卖流程结束后，通常于伦敦时间上午11点前，对每美元金盎司的早盘定价以美元进行报价并公布；或为根据“单位信托及互惠基金守则”的相关规定可接受及批准的任何基准价格。该价格称为早盘LBMA黄金价格。该基金为实物资产ETF。83168.HK仅有人民币柜台，以人民币结算。截至2018年11月28日，恒生人民币黄金ETF市值为4336.33万人民币，过去22日日均交易额为4.25万人民币。该基金管理人为恒生投资管理有限公司，基金托管人为英国汇丰银行有限公司。该基金总开支比率为2.59%。



In [49]:
# delete part of saved dataframe if needed: etf_desc_df

etf_desc_df = etf_desc_df[etf_desc_df.region != 'PRCcomm']

In [59]:
# export to excel
filename_etf_desc = '/home/lena/jupyter-notebook/Lena-RA-ETF/PRC_hk_ETF_LIP_description_%s' % source_csv_date + '.xlsx'
writer = pd.ExcelWriter(filename_etf_desc)
etf_desc_df.to_excel(writer, sheet_name = 'PRC_hk_ETF_LIP_desc_%s' % source_csv_date)
writer.save()
print("File saved: %s" % filename_etf_desc)

File saved: /home/lena/jupyter-notebook/Lena-RA-ETF/PRC_hk_ETF_LIP_description_20181128.xlsx
